In [ ]:

%load_ext autoreload 
%autoreload 2
%matplotlib widget
%matplotlib inline
import multiple_planets_gas_acc as code
import functions_pebble_accretion as peb
import numpy as np
from scipy import integrate
from functions import *
from functions_pebble_accretion import *
import functions_plotting as plot
from matplotlib import cm, ticker
from matplotlib import colors
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import astropy.units as u
import numpy as np
import pandas as pd
import matplotlib.cm as cm
from tqdm.notebook import tqdm
import matplotlib.gridspec as gridspec
import cProfile 
from matplotlib.ticker import ScalarFormatter, LogFormatter, LogLocator


color = mpl.colormaps["YlOrRd"].reversed()(np.linspace(0, 0.7, code.sim_params.nr_planets))


In [ ]:
params_dict = {'St_const': None, 
                'M_dot_star': None,
                #'M_dot_star': (1e-8*const.M_sun.cgs/u.yr).to(u.g/u.s),
                'iceline_radius': None,
                'alpha': 1e-2,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                'iceline_alpha_change': False,
                'iceline_flux_change': False,
                'gas_accretion': False
                }
t_initial = 0.1
a_p0 = np.geomspace(50, 1e-1, num = 50)
t0= ([t_initial] * np.ones(len(a_p0))) # warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work
t_in = (t_initial) #is needed to start the simulation at the right time?
sim_params_dict = {'N_step': 10000,
                'a_p0':a_p0,
                't0':t0,
                't_in':t_in
                }

epsilon_el = [1, 1e-2, 1e-2]
epsilon_h = [1, 0.5, 0.5]

params_lam_ext = code.Params(**params_dict, H_r_model='Lambrechts_mixed', epsilon_el=epsilon_el[0], epsilon_heat=epsilon_h[0])
params_lam = code.Params(**params_dict, H_r_model='Lambrechts_mixed', epsilon_el=epsilon_el[1], epsilon_heat=epsilon_h[1])
params_irr = code.Params(**params_dict, H_r_model='irradiated', epsilon_el=epsilon_el[2], epsilon_heat=epsilon_h[2])
peb_acc = code.PebbleAccretion(simplified_acc=False)
gas_acc = code.GasAccretion()

m0_lam_ext = M0_pla(a_p0, t_in, sigma_gas_steady_state(a_p0, t_in, params_lam_ext), params_lam_ext)
sim_params_lam_ext = code.SimulationParams(**sim_params_dict, m0=m0_lam_ext)
m0_lam = M0_pla(a_p0, t_in, sigma_gas_steady_state(a_p0, t_in, params_lam), params_lam)
sim_params_lam = code.SimulationParams(**sim_params_dict, m0=m0_lam)
m0_irr = M0_pla(a_p0, t_in, sigma_gas_steady_state(a_p0, t_in, params_irr), params_irr)
sim_params_irr = code.SimulationParams(**sim_params_dict, m0=m0_irr)

sim_lam_ext = code.simulate_euler(migration=False, filtering=False, peb_acc=peb_acc,gas_acc = gas_acc,  params=params_lam_ext, sim_params=sim_params_lam_ext)
sim_lam = code.simulate_euler(migration=False, filtering=False, peb_acc=peb_acc, gas_acc = gas_acc, params=params_lam, sim_params=sim_params_lam)
sim_irr = code.simulate_euler(migration=False, filtering=False, peb_acc=peb_acc, gas_acc = gas_acc, params=params_irr, sim_params=sim_params_irr)


In [ ]:
labels = ['Irradiated', 'Surface heating', 'Midplane heating']
fig, axs = plt.subplots(1,3, figsize = (23,8))
num = 10000
clrs = ['grey','black']

# binning the mass on the y-axis between the min and max of the starting and final masses
m_min_lam = M0_pla(sim_lam.position[:,0].value, sim_params_lam.t_in, sigma_gas_steady_state(sim_lam.position[:,0].value, sim_params_lam.t_in, params_lam), params_lam).min()
m_min_lam_ext = M0_pla(sim_lam_ext.position[:,0].value, sim_params_lam_ext.t_in, sigma_gas_steady_state(sim_lam_ext.position[:,0].value, sim_params_lam_ext.t_in, params_lam_ext), params_lam_ext).min()
m_min_irr = M0_pla(sim_irr.position[:,0].value, sim_params_irr.t_in, sigma_gas_steady_state(sim_irr.position[:,0].value, sim_params_irr.t_in, params_irr), params_irr).min()

m_max_lam = M_peb_iso(sim_lam.position[:,0].value, sim_params_lam.t_in, params_lam).max()
m_max_lam_ext = M_peb_iso(sim_lam_ext.position[:,0].value, sim_params_lam_ext.t_in, params_lam_ext).max()
m_max_irr = M_peb_iso(sim_irr.position[:,0].value, sim_params_irr.t_in, params_irr).max()

M_in_min = min(m_min_lam, m_min_lam_ext, m_min_irr)
M_fin_max = max(m_max_lam,m_max_lam_ext,m_max_irr) 
m_bins = np.geomspace(M_in_min, M_fin_max, num = num)
#print("mass binning", m_bins.to(u.M_earth))
Z = [np.zeros((len(m_bins), sim_params_irr.nr_planets))*u.Myr, np.zeros((len(m_bins), sim_params_irr.nr_planets))*u.Myr, np.zeros((len(m_bins), sim_params_irr.nr_planets))*u.Myr]
Zmin = np.array([])*u.Myr
Zmax = np.array([])*u.Myr
for i,(sim, params, sim_params) in enumerate(zip([sim_irr, sim_lam, sim_lam_ext],[params_irr, params_lam, params_lam_ext], [sim_params_irr, sim_params_lam, sim_params_lam_ext])):

    for p in range(sim_params.nr_planets): #loop over the nr_planets
        for m in range(len(m_bins)): #loops over the mass of the planet (timesteps)
            t_idx = np.argmax(sim.mass[p].value >= m_bins[m])
            Z[i][m,p] = sim.time[t_idx].to(u.Myr)
    #axs[i].loglog(sim.position[:,0].to(u.au), M_peb_iso(sim.position[:,0], sim_params.t_in, params).to(u.M_earth), 
            #label = "M_peb_iso", color = 'slateblue', linestyle = ':') #caveat-> this is the M_iso at initial time, for the viscous case it decreases with time
    axs[i].loglog(sim.position[:,0], M_peb_iso(sim.position[:,0].value, sim_params.t_fin, params), 
            label = "M_peb_iso", color = 'slateblue', linestyle = ':') #caveat-> this is the M_iso at final time

    axs[i].loglog(sim.position[:,0], M0_pla(sim.position[:,0].value, sim_params.t_in, 
                                                sigma_gas_steady_state(sim.position[:,0].value, sim_params.t_in, params), params), 
                                                label = "M_in", color = 'lightblue', linestyle = ':')
    #axs[i].loglog(sim.position[:,0].to(u.au), M_3D_2DH_trans(sim.position[:,0].to(u.au), sim_params.t_in, params).to(u.M_earth).value, color = 'lightgrey', linestyle = ':') #caveat, this is the transition mass for t_in, for viscous case it is time dependent
    #axs[i].loglog(sim.position[:,0].to(u.au), M_3D_2DH_trans(sim.position[:,0].to(u.au), sim_params.t_fin, params).to(u.M_earth).value, color = 'lightgrey', linestyle = ':') #caveat, this is the transition mass for t_in, for viscous case it is time dependent
    
    R_irr_visc_in = plot.r_visc_irr(sim.position[:,0].value, sim_params.t_in, params, sim_params)
    R_irr_visc_fin = plot.r_visc_irr(sim.position[:,-1].value, sim_params.t_fin, params, sim_params)
    for j,r in enumerate([R_irr_visc_in, R_irr_visc_fin]):
        col = clrs[j]
        if r is not None:
            print("transition radii"+str(params.H_r_model),r)
            axs[i].axvline(r, ymin = 0, ymax = 0.06, color = col,  alpha = 0.8, linewidth = 2)
            arrow_dx = r/2.5
            axs[i].annotate("", xy=(r-arrow_dx, 2e-6), xytext=(r, 2e-6), 
                         size="large",horizontalalignment="center", verticalalignment="center",
                         arrowprops = dict(arrowstyle = '->', color=col,alpha = 0.6, lw = 2))

    """
    # Filling of the pebble isolation mass
    axs[i].fill_between(sim.position[:,0].to(u.au).value, M_peb_iso(sim.position[:,0], sim_params.t_in, params).to(u.M_earth).value,
                         M_peb_iso(sim.position[:,0], sim_params.t_fin, params).to(u.M_earth).value,  color='slateblue', facecolor='none', hatch='/',alpha=0.3)
    """
    Zmin = np.append(Zmin, Z[i].min())
    Zmax = np.append(Zmax, Z[i].max())

levs = np.geomspace(sim_params.t_in,sim_params.t_fin, num = 20)
vmin = sim_params.t_in
vmax = sim_params.t_fin
for i,(sim, params, sim_params) in enumerate(zip([sim_irr, sim_lam, sim_lam_ext],[params_irr, params_lam, params_lam_ext], [sim_params_irr, sim_params_lam, sim_params_lam_ext])):

    X,Y = np.meshgrid(sim.position[:,0], m_bins)

    #plot white for the regions where the planet is dead or non existent
    Z[i][Z[i].value<=1e-1] = np.nan
    cmap = mpl.colormaps["inferno"].reversed()
    cmap.set_bad(color='white')
    masked_data = np.ma.masked_invalid(Z[i].value)
    print(masked_data)
    CS = axs[i].contourf(X, Y, masked_data, levels= levs, norm=colors.LogNorm(vmin = vmin, vmax = vmax), boundaries=levs, cmap = cmap)

#handling the colorbar	
#fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.95, 0.15, 0.03, 0.7])
cbar = fig.colorbar(CS, cax=cbar_ax)	
# Manually set the colorbar boundaries and ticks
cbar_ax.yaxis.set_major_locator(LogLocator(base=10.0, subs=[1.0, 5.0]))
cbar_ax.yaxis.set_minor_locator(LogLocator(base=10.0, subs='auto'))
cbar_ax.yaxis.set_major_formatter(LogFormatter())
cbar.ax.yaxis.set_major_formatter(plt.FuncFormatter(plot.custom_log_formatter))

cbar.set_label('accretion timescale [Myr]', fontsize=20, labelpad=15)
cbar.ax.tick_params(axis = 'both', which = 'major', size = 13, labelsize = 13)
cbar.ax.tick_params(axis = 'both', which = 'minor', size = 9)

for i in range(3):
    # Set the title and axis labels
    axs[i].set_title(labels[i], fontsize = 25)
    axs[i].set_ylim(M_in_min.to(u.M_earth).value, M_fin_max.to(u.M_earth).value)
    axs[i].set_xlabel('r [AU]', fontsize = 25, labelpad=20)
    axs[i].tick_params(axis='both', which='major', size=15)
    axs[i].tick_params(axis='both', which='minor', size=10)
    axs[i].tick_params(axis='both', which='major', labelsize=15)
    axs[i].figure.axes[-1].yaxis.label.set_size(22)
    axs[i].figure.axes[-1].tick_params(axis='both', which='minor', labelsize=10)
    axs[i].figure.axes[-1].tick_params(axis='both', which='major', labelsize=15)
    axs[i].set_ylim(1e-6, 1e2)
    plot.all_y_ticks(axs[i], 100)


#panel 1
axs[0].text(sim.position[30,0].to(u.au).value, 1.5e-4, 'initial mass', fontsize=18, color='lightblue', rotation=36, rotation_mode='anchor', ha='left', va='bottom')
axs[0].text(sim.position[30,0].to(u.au).value, 3.5, 'pebble isolation mass', fontsize=18, color='slateblue', rotation=19, rotation_mode='anchor', ha='left', va='bottom')
#panel 2
axs[1].text(sim.position[30,0].to(u.au).value, 5e-5, 'initial mass', fontsize=18, color='lightblue', rotation=40.5, rotation_mode='anchor', ha='left', va='bottom')
axs[1].text(sim.position[30,0].to(u.au).value, 3.5, 'pebble isolation mass', fontsize=18, color='slateblue', rotation=19, rotation_mode='anchor', ha='left', va='bottom')
#panel 3
axs[2].text(sim.position[30,0].to(u.au).value, 3e-5, 'initial mass', fontsize=18, color='lightblue', rotation=40.5, rotation_mode='anchor', ha='left', va='bottom')
axs[2].text(sim.position[28,0].to(u.au).value, 3.5, 'pebble isolation mass', fontsize=18, color='slateblue', rotation=19, rotation_mode='anchor', ha='left', va='bottom')

axs[0].set_ylabel('M [$M_{\oplus}$]', fontsize = 25, labelpad=20)
plt.savefig("figures/heatmaps/models_vfrag1", bbox_inches='tight')
